In [ ]:
import os
import csv
import time
import folium
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import concurrent.futures
from sklearn.cluster import KMeans
from geopy.distance import geodesic
from typing import List, Dict, Tuple, Generator

In [ ]:
####### Global performance def #######

"""

The purpose of this code is to define a decorator function called measure_time that 
can be used to measure the execution time of other functions. By applying this decorator 
to a function, you can easily track how long it takes to execute.

When the measure_time decorator is applied to a function, it wraps the original 
function with a wrapper function. The wrapper function checks if the measurement is enabled. 
If it is, the wrapper function records the start time, calls the original function, 
records the end time, calculates the execution time, and prints the results. 
Finally, it returns the result of the original function.


"""
def measure_time(func):
    # Get the enabled value outside the wrapper
    enabled = measure_time.enabled  

    def wrapper(*args, **kwargs):
        # ensure we're using the latest settings
        enabled = measure_time.enabled  
        if enabled:
            
            start_time = time.time()  
            # Call the original function with the provided arguments
            result = func(*args, **kwargs)  
            end_time = time.time()  
            execution_time = end_time - start_time 
            
            print(f" the Function: {func.__name__} took: {execution_time} seconds") 
            return result  # Return the result of the decorated function
        else:
            # If the decorator is disabled, simply call the original function
            return func(*args, **kwargs)  

    # Return the wrapper function only if enabled, else return the original function
    return wrapper if enabled else func

measure_time.enabled = True 

####### ####### ####### ####### #######

In [ ]:
####### Global ploting system #######

"""

The purpose of this code is to define a function called plot_cities that creates a map 
with markers and lines to visualize city data. The function takes 
two parameters: city_data, which contains the information about the cities to be 
plotted, and bound (optional), which specifies the order of cities to be plotted.

The plot_cities function contains several helper functions:

* arrange_cities_nearest: This function arranges the cities in city_data in the order 
specified by path. It sorts the cities based on their index in the path list, ensuring 
that the cities are plotted in the desired order.

* create_map_center: This function extracts the coordinates of the first city in 
city_data and returns them as the center of the map.

* create_custom_icon_style: This function defines a custom icon style for the markers 
on the map. It sets the background color, text color, border radius, padding, font weight, 
and font size.

* add_marker_with_icon: This function adds a marker with a custom icon to the map. 
It takes the coordinates, name, icon style, and index as parameters and creates a marker 
with a numbered icon at the specified coordinates.

* draw_line_between_cities: This function draws a line between two cities on the map. 
It takes the coordinates of the current city, the coordinates of the next city, the names 
of both cities, and the index as parameters. It creates a polyline connecting the two cities with 
a tooltip indicating the route number and the city names.

"""

color = ["Green", "Blue", "Yellow", "Orange", "Purple", "Pink", "Brown", "Gray", "Black", "White"]

def plot_cities(city_data, bound=None, color='blue', map_obj=None):
    
    def arrange_cities_nearest(city_data, path):
        city_names = city_data[:, -1]
        sorted_indices = np.argsort([path.index(city) for city in city_names])
        sorted_city_data = city_data[sorted_indices]
        return np.vstack((sorted_city_data, sorted_city_data[0]))

    def create_map_center():
        return [float(city_data[0][2]), float(city_data[0][3])]

    def create_custom_icon_style():
        return """
            background-color: #ff5959;
            color: #ffffff;
            border-radius: 100%;
            padding: 20%;
            text-align: center;
            font-weight: bold;
            font-size: auto;
        """

    def add_marker_with_icon(coords, name, icon_style, i):
        folium.Marker(
            coords,
            popup=name,
            icon=folium.DivIcon(
                icon_size=(24, 24),
                icon_anchor=(12, 12),
                html='<div style="{}">{}</div>'.format(icon_style, i + 1)
            )
        ).add_to(map_obj)

    def draw_line_between_cities(coords, next_coords, name, next_city, i):
        folium.PolyLine(
            [coords, next_coords],
            color=color,
            weight=2.5,
            opacity=1.0,
            tooltip='Route {}: {} -> {}'.format(i + 1, name, next_city[4])
        ).add_to(map_obj)

    # Create a map if not provided
    if map_obj is None:
        map_center = create_map_center()
        map_obj = folium.Map(location=map_center, zoom_start=6)

    if bound:
        city_data = arrange_cities_nearest(city_data, bound)
        # Iterate over the city data and plot markers and lines
        for i in range(len(city_data) - 1):
            city = city_data[i]
            coords = [float(city[2]), float(city[3])]
            name = city[4]

            icon_style = create_custom_icon_style()

            add_marker_with_icon(coords, name, icon_style, i)

            next_city = city_data[i + 1]
            next_coords = [float(next_city[2]), float(next_city[3])]

            draw_line_between_cities(coords, next_coords, name, next_city, i)
    else:
        # Iterate over the city data and create a folium marker for each city
        for city in city_data:
            folium.Marker(
                location=[float(city[2]), float(city[3])],
                popup=city[4]
            ).add_to(map_obj)

    # Display the map
    return map_obj

####### ####### ####### ####### #######

# City Generator 

this part contains the folowing logic: we first retrieve data from a dataset and later construct a sample from it that contain the cities name, the ZIP Code, the population count and longitude|latitude 

In [ ]:
@measure_time
def read_csv_to_tuple(filename: str):
    # Read the CSV file using pandas
    df = pd.read_csv(filename, delimiter=';', encoding='ISO-8859-1')
    # Convert the DataFrame to a tuple of lists and then to a tuple
    cities = tuple(df.values.tolist())  
    return cities


@measure_time
def sample_N_from_tuple(cities: tuple, size: int = None):
    # Create a DataFrame from the tuple of lists
    df = pd.DataFrame(list(cities))
    # Get the total number of rows in the DataFrame
    totalRows = len(df)
    
    # If size is not specified or greater than totalRows
    # Return an empty tuple
    if size is None or size > totalRows:  
        return ()
    
    # Randomly select 'size' indices without replacement
    indices = np.random.choice(totalRows, size, replace=False)
    # Extract the sampled data based on the selected indices
    sampled_data = df.iloc[indices].values.tolist()  
    # Return the sampled data as a NumPy array
    return np.array(sampled_data)  

# location generator 
The purpose of this staged is to generate a series of city names along with their respective longitude and latitude coordinates. It achieves this by extracting the relevant information from a given list of city data

In [ ]:
@measure_time
def create_location_generator(citiesSample: List[List[str]]) -> Generator[Tuple[str, Tuple[float, float]], None, None]:
    # Extract city names from citiesSample using NumPy array indexing
    city_names = np.array(citiesSample)[:, 4]
    # Extract longitudes and convert them to float using NumPy array indexing
    longitudes = np.array(citiesSample)[:, 3].astype(float)
    # Extract latitudes and convert them to float using NumPy array indexing
    latitudes = np.array(citiesSample)[:, 2].astype(float)

    yield from zip(city_names, zip(longitudes, latitudes))

# misc 

In [ ]:
def pprint_the_output(best_time, best_route): 
    print(f"Best time: {best_time} h")
    print("Best Route:", ' -> '.join(best_route))